In [1]:
import pandas as pd
import numpy as np
import unidecode
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from textblob import TextBlob
#from googletrans import Translator
#from translate import Translator
#import indicoio

In [2]:
data = pd.read_csv("Data\\preguntas.csv", sep=',', encoding = "ISO-8859-1")
data.head(3)

,Pregunta,respuesta,Tema,Pers/empresa,Polarity
0,Hola una pregunta en que perjudica el acta de ...,NaN,Renuncia/Despido/Desahucio,persona,0
1,PARA CALCULAR EL DESAHUCIO SE DEBE TOMAR EN CU...,NaN,Renuncia/Despido/Desahucio,persona,0
2,supuestamente Correa emitió el decreto 225 dod...,NaN,Jubilacion Patronal,persona,0


In [3]:
p2 = []
for p in data.Pregunta:
    p2.append(' '.join(p.split()))
data['Pregunta2'] = p2

In [6]:
tokenizer = RegexpTokenizer(r'\w+')
stemmer = SnowballStemmer("spanish")
#translator = Translator()
#indicoio.config.api_key = '6d3578144af03d329e72390017a320cf'
#translate = YandexTranslate('trnsl.1.1.20190627T002314Z.7aaae22624849593.80a9f268fd6e46bda2a31159dab873b5687841d4')

In [7]:
jbtopdict = np.load('Data\\jbtopdict.npy').item()
rentopdict = np.load('Data\\rentopdict.npy').item()
IESStopdict = np.load('Data\\IESStopdict.npy').item()
CONTtopdict = np.load('Data\\CONTtopdict.npy').item()
OStopdict = np.load('Data\\OStopdict.npy').item()
CONStopdict = np.load('Data\\CONStopdict.npy').item()
JStopdic = np.load('Data\\JStopdict.npy').item()

In [8]:
def top_likelihood(t_dict, sentence):
    lik = -1
    tokens = tokenizer.tokenize(unidecode.unidecode(sentence.lower()))
    for w in tokens:
        w = stemmer.stem(w)
        if w in t_dict:
            #print(w)
            if lik == -1:
                if t_dict[w] == 0:
                    lik = -0.9
                else:
                    lik = t_dict[w]
            else:
                if t_dict[w] == 0:
                    lik -= 0.9
                else:
                    lik += t_dict[w]
    return lik

In [9]:
data.Pregunta2[62]

'le saluda Miguel Zamora le acabe de enviar con correo a la Ventas3 favor para que lea mi correo y me conteste gracias'

In [10]:
top_likelihood(jbtopdict , data.Pregunta2[62])

0.5273504273504271

In [11]:
stemmer.stem('contactaria')

'contactari'

In [12]:
#from yandex_translate import YandexTranslate
#translate = YandexTranslate('trnsl.1.1.20190627T002314Z.7aaae22624849593.80a9f268fd6e46bda2a31159dab873b5687841d4')
#print('Translate:', translate.translate('Hola soy Roberto', 'es-en'))

In [13]:
def sentiment(sentence):
    return pass

In [15]:
#sentiment('Hola me llamo roberto')

In [16]:
def percent_greet(sentence):
    tgreet = ['hol', 'buen', 'tard', 'dias', 'noch']
    count = 0
    tokens = tokenizer.tokenize(unidecode.unidecode(sentence.lower()))
    for w in tokens:
        w = stemmer.stem(w)
        if w in tgreet:
            count += 1            
    return count/len(tokens) 

In [17]:
def n_token(sentence):
    tokens = tokenizer.tokenize(unidecode.unidecode(sentence.lower()))       
    return len(tokens) 

In [18]:
data['jplik'] = [top_likelihood(jbtopdict, x) for x in data.Pregunta2]
data['renlik'] = [top_likelihood(rentopdict, x) for x in data.Pregunta2]
data['IESSlik'] = [top_likelihood(IESStopdict, x) for x in data.Pregunta2]
data['CONTlik'] = [top_likelihood(CONTtopdict, x) for x in data.Pregunta2]
data['OSlik'] = [top_likelihood(OStopdict, x) for x in data.Pregunta2]
data['CONSlik'] = [top_likelihood(CONStopdict, x) for x in data.Pregunta2]
data['JSlik'] = [top_likelihood(JStopdic, x) for x in data.Pregunta2]

In [19]:
#data['polarity'] = [sentiment(x) for x in data.Pregunta2]

In [20]:
data['p_greet'] = [percent_greet(x) for x in data.Pregunta2]

In [21]:
data['n_token'] = [n_token(x) for x in data.Pregunta2]

In [22]:
data.head(3)

,Pregunta,respuesta,Tema,Pers/empresa,Polarity,Pregunta2,jplik,renlik,IESSlik,CONTlik,OSlik,CONSlik,JSlik,p_greet,n_token
0,Hola una pregunta en que perjudica el acta de ...,NaN,Renuncia/Despido/Desahucio,persona,0,Hola una pregunta en que perjudica el acta de ...,-1.657143,2.857143,-2.700000,-2.700000,-2.7,-2.7,-2.700000,0.076923,13
1,PARA CALCULAR EL DESAHUCIO SE DEBE TOMAR EN CU...,NaN,Renuncia/Despido/Desahucio,persona,0,PARA CALCULAR EL DESAHUCIO SE DEBE TOMAR EN CU...,1.282051,1.439103,-0.621154,-2.700000,-2.7,-2.7,-2.700000,0.000000,17
2,supuestamente Correa emitió el decreto 225 dod...,NaN,Jubilacion Patronal,persona,0,supuestamente Correa emitió el decreto 225 dod...,4.085470,-2.546154,-2.494872,-3.155556,-4.5,-4.5,-3.488889,0.000000,30


In [23]:
data.to_csv("Data\\preguntas2.csv", index=False)